In [146]:
import requests
import bs4
from lxml import html
from bs4 import BeautifulSoup
from distutils.filelist import findall
import requests
import re
import pandas as pd

def Walmart(SKU, ZIP):
    url = 'https://brickseek.com/walmart-inventory-checker/'
    payload = {'search_method': 'sku', 'sku': SKU, 'zip': ZIP, 'sort': 'distance'}
    df_record = pd.DataFrame(columns=['Store','City','Availability','Quantity'])
    r = requests.post(url, data=payload).text    # Make a POST request with data


    bs = BeautifulSoup(r, 'html.parser')
    j=0
    for tag in bs.find_all('div', class_='table__body'): 
        for i in range(10):
            m_Store = tag.findAll('strong', class_='address-location-name')
            m_Store = tag.findAll('strong', class_='address-location-name')
            m=str(m_Store)
            if i < m.count('/strong'):
                m_s= m_Store[i].get_text().replace("\nWalmart","")
                m_add = tag.findAll('address',class_="address")
                m_Address = m_add[i].contents[2]      
                m_Availability = tag.findAll('span',class_="availability-status-indicator__text")
                m_a = m_Availability[i].get_text()
                if m_a =='Out of Stock':
                    m_q = str(0)
                    j=j-1
                else:    
                    m_Quantity = tag.findAll('span',class_="table__cell-quantity")
                    m_q = m_Quantity[j].get_text()[9:]
                j=j+1
                df_record = df_record.append({'Store':m_s, 'City':m_Address, 'Availability':m_a, 'Quantity':m_q }, ignore_index=True)
            else:
                print(df_record)
                if len(df_record) == 0:
                    print("No result found")
                print()
                break

def WalmartKeywords(keywords,ZIP):
    url = "http://www.walmart.com/search/?query="
    r = requests.post(url+keywords).text
    bs = BeautifulSoup(r, 'html.parser')
    result = re.findall(r'href="/ip/(.*?)"', str(bs))
    result = [item for item in result if not("#customer-reviews" in item)]
    result = list(dict.fromkeys(result))
    df = pd.DataFrame(result)
    df.columns = {"product"}
    df[['product','sku']] = df['product'].str.split("/",expand=True) 
    df['product'] = df['product'].str.replace("-"," ")

    for resultitem in range(0,len(df)):
        print(str(resultitem) + ": " + df['product'][resultitem])
    print()
    print("Type in the index (number) of your requested item: ")
    requestitem = int(input())
    Walmart(df['sku'][requestitem],ZIP)
    return()



def Target(SKU, ZIP):
    SKU=str(SKU)
    if '-' not in SKU:
        SKU = ('{}-{}-{}'.format(SKU[0:3], SKU[3:5], SKU[5:9]))
    url = 'https://brickseek.com/target-inventory-checker/'
    payload = {'search_method': 'sku', 'sku': SKU, 'zip': ZIP, 'sort': 'distance'}

    r = requests.post(url, data=payload).text    # Make a POST request with data

    bs = BeautifulSoup(r, 'html.parser')
    print(" Store                Availability      Quantity ")
    j=0
    for tag in bs.find_all('div', class_='table__body'): 
        for i in range(10):
            #print(tag)
            m_Store = tag.findAll('strong', class_='address-location-name')
            m=str(m_Store)
            if i < m.count('/strong'):
                m_s= m_Store[i].get_text()
                m_add = tag.findAll('address',class_="address")
                m_Address = m_add[i].contents[0]      
                m_Availability = tag.findAll('span',class_="availability-status-indicator__text")
                m_a = m_Availability[i].get_text()
                m_q = 'Unknown'
                j=j+1
                print( m_s+"   "  +  m_Address   + "      " + m_a + "        " + m_q ) 
            else:break

In [151]:
WalmartKeywords("Lego",96848)

0: LEGO Creator Expert Bookshop 10270 Modular Building Kit Big LEGO Set and Collectors Toy for Adults 2 504 Pieces
1: LEGO Technic Fast Furious Dom s Dodge Charger 42111 Race Car Building Set 1077 Pieces
2: LEGO City Passenger Train nbsp 60197
3: LEGO Minecraft The Blaze Bridge 21154 Toy Battle Building Set 370 Pieces

Type in the index (number) of your requested item: 
1


()

### question above: return nothing when selecting some items in the shop

In [142]:
WalmartKeywords("Nintendo",96848)

0: Nintendo Switch Lite Console Turquoise
1: Nintendo 2DS System with Super Mario Maker Pre Installed Yellow Red FTRSYBDW
2: Super Mario Odyssey Nintendo Nintendo Switch 045496590741
3: Nintendo Switch Gaming Console Neon Blue and Neon Red Joy Con Bundle with Mario Kart Deluxe 8
4: Nintendo Switch Bundle with Mario Kart 8 Deluxe Gray
5: Nintendo Universal Super NES Classic Edition
6: Nintendo Switch Joy Con Pair Neon Pink and Neon Green
7: NES Classic Mini EU Console
8: Animal Crossing New Horizons Nintendo Nintendo Switch 045496596439
9: Super Nintendo Entertainment System SNES Classic Edition

input your requested item index: 
0
            Store                  City  Availability Quantity
0  Walmart #3478     Honolulu HI 96814   Out of Stock        0
1  Walmart #3149     Honolulu HI 96813   Out of Stock        0
2  Walmart #5274   Pearl City HI 96782   Out of Stock        0
3  Walmart #2314      Waipahu HI 96797   Out of Stock        0
4  Walmart #2126     Mililani HI 96789   Out o

()

## Below are tests

In [48]:
import requests
import re
import pandas as pd

In [7]:
url = "http://www.walmart.com/search/?query="
r = requests.get(url+"nintendo")

In [15]:
r = requests.post(url+"nintendo").text

In [17]:
bs = BeautifulSoup(r, 'html.parser')

In [35]:
#result = re.findall(r'productPageUrl":"/ip/(.*?)","', str(bs))

In [41]:
url = "http://www.walmart.com/search/?query="
r = requests.post(url+keywords).text
bs = BeautifulSoup(r, 'html.parser')
result = re.findall(r'href="/ip/(.*?)"', str(bs))
result = [item for item in result if not("#customer-reviews" in item)]
result = list(dict.fromkeys(result))
df = pd.DataFrame(result)
df.columns = {"product"}
df[['product','sku']] = df['product'].str.split("/",expand=True) 
df['product'] = df['product'].str.replace("-"," ")


In [43]:
result = list(dict.fromkeys(result))

In [73]:
df = pd.DataFrame(result) 

In [74]:
df.columns = {"product"}

In [76]:
df[['product','sku']] = df['product'].str.split("/",expand=True) 

In [78]:
df['product'] = df['product'].str.replace("-"," ")

In [85]:
df['sku'][9]

'55432571'

In [120]:
result[5]

'Nintendo-Universal-Super-NES-Classic-Edition/410336025'

In [99]:
dftest = pd.DataFrame(columns = ['a','b','c'])

In [103]:
dftest.append()

,a,b,c
